In [0]:
# import libraries
import altair as alt
import pandas as pd

In [0]:
# it just needs to be here so we can work with big files 

alt.data_transformers.enable('default', max_rows=None) 

DataTransformerRegistry.enable('default')

On our first session we reproduced famous Gapminder plot. Well, at least part of this plot (you can find my code here). Now you have to add more data and some interactivity to it.

1. Whenever you mouseover / hover on some country on the chart, you should be able to see other countries that belong to the same continent highlighted (so other countries should have less opacity).

2. You should have slider under the chart and be able to filter the year with it (please notice that first we recreated the chart just for one year but now you have to use extended version of this dataset with more years provided).
3. You should add a small side map colored by continent just as original chart does (open the page and click "color" button to see it). This map should work both as a legend and a filter - so whenever you click on some country on the chart, you should be able to see other countries that belong to the same continent highlighted. For this task you might need to use world.geojson. And you probably have to check whether countries/continents are named the same way in the csv and geojson.

In [0]:
df = pd.read_csv('https://gist.githubusercontent.com/rpartsey/277b82a6884f1893ac1b784ff96f6844/raw/c7ba48061ade57bee34a194f0d68818c7bec332a/gapminder.csv')

url_geojson = 'https://gist.githubusercontent.com/rpartsey/5b6a1aff8634d08c4c9d629b910d7f4d/raw/9f7a2d18f24698675c341851ede107fc8df494e7/world.geojson'
data_geojson_remote = alt.Data(url=url_geojson, format=alt.DataFormat(property='features',type='json'))

continents_color_range = ["#00D5E9", "#7FEB00", "#FF5872", "#FFE700"]
slider = alt.binding_range(min=2009, max=2017, step=1, name='year:')
slider_selection = alt.selection_single(name="year_selector", fields=['year'],bind=slider, init={'year': 2017})
single_selection = alt.selection_single(fields=['continent'])
mouse_hover = alt.selection(type='single', on='mouseover', fields=['continent'])

gdp_per_cap_chart = alt.Chart(df).mark_circle().encode(
    x = alt.X('gdpPerCap:Q', 
              title='GDP per capita, $',
              scale=alt.Scale(type='log', base=2),
              axis=alt.Axis(format='.1s')),
    y = alt.Y('lifeExp:Q', 
              title='life expectancy, years',
              scale=alt.Scale(zero=False, bins=[50, 55, 60, 65, 70, 75, 80, 85], domain = [50, 85])),
    color = alt.Color('continent:N', 
                      title='continent',
                      scale=alt.Scale(range=continents_color_range),
                      legend=None
                      ),
    opacity = alt.condition(mouse_hover | single_selection, alt.value(0.9), alt.value(0.2)),
    tooltip = alt.Tooltip(shorthand=['continent:N', 'country:N']),
    size = alt.Size('pop:Q', 
                    title='population',
                    scale=alt.Scale(range=[10, 1500]),
                    legend=None)
).add_selection(mouse_hover).add_selection(slider_selection).transform_filter(slider_selection)

continents_map_chart = alt.Chart(data_geojson_remote, title="World regions").mark_geoshape().encode(
    color=alt.Color('continent:N', 
                    title='continent',
                    scale=alt.Scale(range=continents_color_range),
                    legend=None),
    opacity = alt.condition(single_selection, alt.value(0.9), alt.value(0.2)),
    tooltip = alt.Tooltip(shorthand=['continent:N'])
).transform_calculate(
    continent = 'datum.properties.continent'
).properties(width=150, height=150).add_selection(single_selection)


alt.hconcat(gdp_per_cap_chart, continents_map_chart).configure_axis( 
    domain = False, 
    ticks = False,
    labelPadding = 10,
    labelFont = 'Ubuntu Mono',
    labelFontSize = 12,
    labelColor = '#3A3F4A',
    titleFont = 'Ubuntu Mono',
    titleColor = '#3A3F4A',
    titleAnchor = "end",
    titlePadding = -40,
    gridWidth = 0.5,
    gridDash = [1, 0, 1]
).configure_title(
    color="#3A3F4A",
    font='Ubuntu Mono',
    fontSize=12
).configure_view(
    strokeWidth = 0,
    width = 850
)

alt.HConcatChart(...)